In [1]:
import pandas as pd
import numpy as np
import os, cv2, json
from tqdm import tqdm
from sklearn.utils import shuffle

In [2]:
class CFG:
    data_dir = './data'
    train_dir = os.path.join(data_dir, 'train_images')
    img_size = (720, 1280)
    
cfg = CFG()

In [3]:
train_df = pd.read_csv('./data/train.csv')
train_df.head()

,video_id,sequence,video_frame,sequence_frame,image_id,annotations
0,0,40258,0,0,0-0,[]
1,0,40258,1,1,0-1,[]
2,0,40258,2,2,0-2,[]
3,0,40258,3,3,0-3,[]
4,0,40258,4,4,0-4,[]


In [ ]:
def draw_frame(frame_data):
    _, _, _, _, image_id, annots = frame_data
    video_num, frame_num = image_id.split('-')
    
    fpath = os.path.join(cfg.train_dir, f'video_{video_num}', f'{frame_num}.jpg')
    img = cv2.imread(fpath)
    annots = json.loads(annots.replace("'", '"'))
    
    for annot in annots:
        xmin = int(annot['x'])
        ymin = int(annot['y'])
        xmax = int(annot['x']) + int(annot['width'])
        ymax = int(annot['y']) + int(annot['height'])
        img = cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)
        
    cv2.imshow('img', img)
    cv2.waitKey()
    cv2.destroyAllWindows()
    

draw_frame(train_df.iloc[57].values)

In [8]:
def create_yolo5_dataset(train_df):
    df = shuffle(train_df)
    thresh = int(df.shape[0] * 0.8)
    for x in tqdm(range(df.shape[0])):
        if x <= thresh:
            train_val = 'train'
        else:
            train_val = 'val'
        
        frame_data = df.iloc[x]
        _, _, _, _, image_id, annots = frame_data
        video_num, frame_num = image_id.split('-')

        fpath = os.path.join(cfg.train_dir, f'video_{video_num}', f'{frame_num}.jpg')
        img = cv2.imread(fpath)
        annots = json.loads(annots.replace("'", '"'))
        
        new_img_path = os.path.join(f'/home/sm00th/Projects/kaggle/great_barrier_reef/yolo_dataset/images/{train_val}', f'{video_num}_{frame_num}.jpg')
        new_annot_path = os.path.join(f'/home/sm00th/Projects/kaggle/great_barrier_reef/yolo_dataset/labels/{train_val}', f'{video_num}_{frame_num}.txt')
        
        if annots != []:
            with open(new_annot_path, 'a') as txt:
                for annot in annots:
                    xmin = int(annot['x'])
                    ymin = int(annot['y'])
                    xmax = int(annot['x']) + int(annot['width'])
                    ymax = int(annot['y']) + int(annot['height'])

                    xc = (int(annot['x']) + int(annot['width']) / 2) / cfg.img_size[1]
                    yc = (int(annot['y']) + int(annot['height']) / 2)  / cfg.img_size[0]
                    w = int(annot['width']) / cfg.img_size[1]
                    h = int(annot['height']) / cfg.img_size[0]

                    data = f'0 {str(xc)} {str(yc)} {str(w)} {str(h)}\n'
                    txt.write(data)
                    
            cv2.imwrite(new_img_path, img)

In [9]:
create_yolo5_dataset(train_df)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 23501/23501 [10:51<00:00, 36.09it/s]
